Start this first
```bash
$ litex_server uart /dev/ttyUSB0 1152000
```

In [217]:
%pylab notebook
from litex.soc.tools.remote import RemoteClient
from litescope.software.driver.analyzer import LiteScopeAnalyzerDriver
from struct import pack, unpack
import time
from hexdump import hexdump
from subprocess import run

def getPacket(dump, name="__main___mac_rx_cdc_source"):
    """
        dump a binary stream from litexscope capture
        rxn = name of data channel
        vn = name of data valid channel
        fn = name of output binary file
    """
    rxs = dump[name + "_payload_data"][::2]
    vs = dump[name + "_valid"][::2]
    return bytes([rx for v, rx in zip(vs, rxs) if v])

Populating the interactive namespace from numpy and matplotlib


# Connect to hardware

In [218]:
for i in range(32):
    try:
        r = RemoteClient(csr_csv="build/csr.csv", debug=False, port=1234 + i)
        r.open()
        print("Connected to Port", 1234 + i)
        break
    except ConnectionRefusedError:
        pass
"".join([chr(r.read(r.bases.identifier_mem + i * 4) & 0xFF) for i in range(64)]).strip("\x00")

Connected to Port 1234


'SP605 eth demo 2019-04-10 15:56:03'

# Debug clocks

In [222]:
r.regs.f_tx_value.read()

125005216

# Setup litescope

In [223]:
analyzer = LiteScopeAnalyzerDriver(r.regs, "analyzer", config_csv="build/analyzer.csv", debug=True)
analyzer.configure_subsampler(1)
# Trigger on transmitting a packet
analyzer.add_rising_edge_trigger("mac_tx_cdc_sink_valid")

In [224]:
analyzer.run(offset=128, length=256)
analyzer.wait_done()
dump = analyzer.upload()
analyzer.save("dump.vcd")
analyzer.save("dump.py")

[running]...
[uploading]...
|====================>| 100%
[writing to dump.vcd]...
[writing to dump.py]...


Run this to trigger it
```bash
$ ping 192.168.1.50
```

In [ ]:
!gtkwave dump.vcd

# Extract packet data as seen by the FPGA

In [213]:
from dump import dump
with open("dump.hex", "w") as f:
    f.write(hexdump(getPacket(dump, "mac_rx_cdc_source"), "return"))
    f.write("\n")
    f.write(hexdump(getPacket(dump, "mac_tx_cdc_sink"), "return"))
!cat dump.hex
!text2pcap dump.hex dump.pcap
!rm -rf dump.hex

00000000: FF FF FF FF FF FF 34 17  EB BA 05 DE 08 06 00 01  ......4.........
00000010: 08 00 06 04 00 01 34 17  EB BA 05 DE C0 A8 01 2A  ......4........*
00000020: 00 00 00 00 00 00 C0 A8  01 32 00 00 00 00 00 00  .........2......
00000030: 00 00 00 00 00 00 00 00  00 00 00 00 00 00        ..............
00000000: 34 17 EB BA 05 DE 10 E2  D5 00 10 00 08 06 00 01  4...............
00000010: 08 00 06 04 00 02 10 E2  D5 00 10 00 C0 A8 01 32  ...............2
00000020: 34 17 EB BA 05 DE C0 A8  01 2A 00 00 00 00 00 00  4........*......
00000030: 00 00 00 00 00 00 00 00  00 00 00 00              ............Input from: dump.hex
Output to: dump.pcap
Output format: pcap
Wrote packet of 62 bytes.
Wrote packet of 60 bytes.
Read 2 potential packets, wrote 2 packets (178 bytes).


In [214]:
!wireshark dump.pcap

In [ ]:
r.close()